<div style="
    display: flex;
    justify-content: center;
    align-items: center;
    height: 100px;
    border: 4px solid #32CD32;
    border-radius: 15px;
    background-color: #000000;
    box-shadow: 0 4px 8px 0 rgba(255, 215, 0, 0.2);
    margin: 20px;
">
    <h1 style="
        color: #32CD32;
        font-family: 'Nexa','', ;
        font-size: 52px;
        text-shadow: 2px 2px 4px #32CD32;
    ">
        MASSIVEChem 
    </h1>
</div>

## Introduction
The following Jupyter notebook briefly presents MASSIVEChem, a pip-installable python package. This package is developped around mass spectrometry, a tool commonly used in analytical chemistry, biochemistry, pharmaceutical sciences and more. Fundamentally, this package simulates and plots the mass spectrum of a given molecule, with a chosen resolution and with a chosen precision. 

This package was created as a collaborative project for the EPFL course Practical programming in chemistry [![GitHub3](https://img.shields.io/badge/EPFL-CH200-red.svg)](https://edu.epfl.ch/studyplan/en/bachelor/chemistry-and-chemical-engineering/coursebook/practical-programming-in-chemistry-CH-200)

Before going through the code and the use of the package, let's briefly discuss the ideas that allowed to build this package

## Ideas

Initially, the plan for this project was to write a code that would predict the NMR spectrum of an input molecule. 3 weeks were spent trying to figure out a way to analyze molecules, to find functional groups, and to predict the shift of the hydrogens in a molecule. As the time was advancing, it became apparent to us that this project was beyond the scope of our abilities as it would most likely require the use of machine learning, a concept that none of us was familiar with. 

Still wanting to create a package that would display some type of visual information from a molecule, our minds collectively turned towards mass spectroscopy. Being familiar with the theoretical concepts behind this analytical method, having followed a course on mass spectroscopy in first year and having revisited the topic in our second-year electromagnetics course, we were more confident in our abilities regarding this idea.

The concept of mass spectrometry consists of ionizing a molecule and passing this molecule in a strong magnetic field. Thanks to the Lorentz force, the molecule is deviated as a function of its mass which allows to separate the different isotopes present in a sample. By  putting a detector at the end, it is possible to determine which molecule was put in the mass spectrometer.



Coding wise, simulating the mass spectrum was done by using list comprehension and combinatory probabilities. This involves a systematic breakdown of the process into discrete steps

- Molecule Definition: First the molecules to be analyzed needs to be defined, specifying its masses and relative abundances. Each molecule can be represented as a data structure (a list) containing its atoms and abundance.

- Fragmentation Simulation: Then, the fragmentation process needs to be simulated. This step involves generating all possible combinations of masses for each atom using the algorithm of probability trees. 

- Mass Calculation: The Mass of each fragment is calculated by summing the masses of the constituent atoms. This step ensures that each fragment is assigned an accurate mass value based on its atomic composition.

- Combination of Fragments: At the end of the mass calculation process, all fragments are combined into a single list and the fragments with similar masses are combined. This aggregated list represents the collective set of fragments resulting from the analyzed molecules.

- Mass Spectrum Simulation: This step involves estimating the abundance of each fragment considering both the relative abundances of the original molecules and the probabilities associated with each generated fragment. 

- Visualization: The mass spectrum can then be plotted using a plotting package. In our case we chose bokeh.

Once that the spectrum was simulated we thought of different feature to add to the spectrum to facilitate its comprehension. First we added an image of the molecule using rdkit.Draw. Then we developped a function to recognise the functional groups present in the molecule and as well as a function to show them. To make the spectrum more interactive, we added a second plot which allows to see where the user is zooming on the principal graph. Finally, we decided to add a function to show an interactive 3D graph of the molecule using xyz2graph package.


## Difficulties

Once we had written the code that would do the actual computation of the different combinations of isotopes that can compose the molecule (with their relative probabilities of existing), it became very noticeable that the time complexity of the algorithm was subpar (being of roughly $2^n$ with n being the number of atoms in the molecule; the code would take ~30s to run for pentane). To remedy this fact, a part of code was implemented that would disregard any molecule with a probability of under $10^{-6}$ of existing (this reduced the time complexity form $2^n$ to n).

Another difficulty encountered was importing data.
The initial project included a data folder for importing the abundances of the various isotopes. Unfortunately, when the package was exported via pypi, the files inside the folder mentioned were not included. Once installed, the package could not read the data files required for the package to function correctly.
Despite various attempts to include these data files in the package, we had to come up with another solution that did not use this folder. As the initial code used the path to this folder to read the isotope abundances, we had to modify the code written so far to integrate the data directly without using an external file. This can be seen at the beginning of the spectrum function. 
In the end, the problem was solved, although this required some 70 extra lines of code, whereas one would normally have sufficed. 

Finding a good tool to create the mass spectrum graph was also a challenge. Various tools such as matplotlib or plotly can be used to create good quality graphs, but none of them met the three criteria we wanted: precision, interactivity and aesthetics. So we had to explore tools that allowed us to do this, and we chose to use Bokeh. It allowed us to generate graphs as we wanted them and with excellent quality. None of us had used it before, so we had to do a lot of research to learn how to use it correctly and effectively.

Another difficulty arose directly from the SMILES, which we used to detect the different functional groups present. We had to create a code that differentiated between the groups, using SMARTS in particular, which enabled us to specify the differences and in particular the bonds.
Take an ester, for example. The system detects a carboxylic acid when it's not one so we had to specify this using the following SMARTS "CC(=O)[Oh0]". Which specifies that the oxygen does not have a bond with a hydrogen.
All the different smarts used are present in the function "functional_group_display" in the scripts folder. 

## Limitations

The complete package does, however, have a few limitations. 
Firstly, pure elements and other molecules such as proteins and polysaccharides do not have their own SMILES. It is therefore impossible to visualise their mass spectra using this package. 
However, we believe that it would be possible to implement these molecules without SMILES using other encoding methods.

The second is obviously time. The more complex the molecule, the more atoms it contains, the longer it will take to run the programme. 

*Example* : 
spectrum(mol_smi, imprecision_True_False, apparatus_resolution)

1) If imprecision_True_False = True, we have a complexity of $n$
2) If imprecision_True_False = False, we have a complexity of $2^{n}$ 

Where n = number of atoms in the molecule

## Usage

Once the package is installed following the README, a whole set of functions will be imported.
For example, to use the function that takes a molecule under SMILEs representation, a precision boolean and an apparatus resolution as inputs, and outputs the spectrum, one has to do the following:

In [ ]:
import MASSiveChem.MASSiveChem as MC
from bokeh.plotting import show, output_notebook
output_notebook()
show(MC.spectrum('CC1(C(N2C(S1)C(C2=O)NC(=O)CC3=CC=CC=C3)C(=O)O)C', True, 0.01))

Note: the second argument here being True indicates that the function will neglect any ions which have a probability of apparation under 0.00001

## Examples of functions and their respective inputs and outputs
First, here is a list of the functions that come with the package: 

## Basic functions
- calculate_unsaturation(mol_smi)->unsaturation

- SMILEs_interpreter(mol_smi)->mol

- molecule_list_generator(mol)->list_atoms

- ionisation_method(list_atoms)->list_atoms

- sulphur_nitrogen_finder(list_atoms)->has_N, has_S, count_N, count_S

- main_function(list_atoms, imprecision_True_false)->list_masses, list_abundances

- peak_merger(list_ions, list_abundances)->list_masses, list_abundances

- sulphur_nitrogen_adder(list_ions, list_abundances, has_N, has_S, count_N, count_S)->list_masses, list_abundances

- peak_sorter(list_masses, list_abundances)->list_masses, list_abundances

- delta_function_plotter(list_masses, list_abundances)->list_masses, list_abundances

- double_plot(list_masses, list_abundances)->bokeh_plot_1, bokeh_plot_2

- functional_group_finder(mol_smi)->list_functional, groups_contained

- functional_group_display(ist_functionalgroups_contained)-> bokeh_plot_3

- mol_web_show(mol_smi)->bokeh_plot_4

- all_in_one(bokeh_plot_1, bokeh_plot_2, bokeh_plot_3)-> bokeh_plot_final

- smiles_to_3D_plot(mol_smi)-> 3D_panel_plot

from all of these functions, the following two main functions are most useful:
## Main functions



- spectrum(mol_smi, imprecision_True_False, apparatus_resolution)

- specturm_3D(mol_smi, imprecision_True_False, apparatus_resolution)

## Functionality
The main functions can both be made by combining the basic functions.

# Examples
First, all of the basic functions will be shown, in order to then construct spectrum and spectrum_3D out of them. The plots are fully interactive, so you can zoom in on the areas you want. 



## SMILEs interpreter
This function takes a molecule under SMILEs representation as input and output its MOL represerntation

In [ ]:
import MASSiveChem.MASSiveChem as MC
print(MC.SMILEs_interpreter('CC1(C(N2C(S1)C(C2=O)NC(=O)CC3=CC=CC=C3)C(=O)O)C'))

## molecule_list_generator
This function takes a MOL representation as input and outputs a list of all of the atoms within that molecule

In [ ]:
import MASSiveChem.MASSiveChem as MC
mol = MC.SMILEs_interpreter('CC1(C(N2C(S1)C(C2=O)NC(=O)CC3=CC=CC=C3)C(=O)O)C')
print(MC.molecule_list_generator(mol))

## ionisation_method(list_atoms)
This function takes a list of atoms as input and simply outputs the same list with a proton less (in essence, a hydrogen atom)

In [ ]:
import MASSiveChem.MASSiveChem as MC

list_atoms = ['C', 'C', 'C', 'N', 'C', 'S', 'C',
            'C', 'O', 'N', 'C', 'O', 'C', 'C',
            'C', 'C', 'C', 'C', 'C', 'C', 'O',
            'O', 'C', 'H', 'H', 'H', 'H', 'H',
            'H', 'H', 'H', 'H', 'H', 'H', 'H',
            'H', 'H', 'H', 'H', 'H', 'H']

print(MC.ionisation_method(list_atoms))

## sulphur_nitrogen_finder(list_atoms)
This function takes a list of atoms as input and output four values: has_N, has_S, count_N and count_S.
has_N & has_S are True if there is an odd number of Nitrogen and Sulphur repsectively, and False otherwise.

In [ ]:
import MASSiveChem.MASSiveChem as MC

list_atoms = ['C', 'C', 'C', 'N', 'C', 'S', 'C',
            'C', 'O', 'N', 'C', 'O', 'C', 'C',
            'C', 'C', 'C', 'C', 'C', 'C', 'O',
            'O', 'C', 'H', 'H', 'H', 'H', 'H',
            'H', 'H', 'H', 'H', 'H', 'H', 'H',
            'H', 'H', 'H', 'H', 'H']

output = (MC.sulphur_nitrogen_finder(list_atoms))
print(output)

## main_function(list_atoms)
This function takes a list of atoms as an input, and will output two lists.

The first (list_masses) is a list of the masses of all of the possible combinations of isotopes that can form the given molecule (under list representation) and the second (list_abundances) is a list of the probabilities of apparition of each of these combinations of isotopes.

As such, both lists are the same length and the mass at index i in list_masses has the probability at index i in list_abundances associated to it.

In [ ]:
import MASSiveChem.MASSiveChem as MC

output = MC.main_function(['C', 'C', 'C', 'N', 'C', 'S', 'C','C', 'O', 'N', 'C', 'O', 'C', 'C',
                            'C', 'C', 'C', 'C', 'C', 'C', 'O','O', 'C', 'H', 'H', 'H', 'H', 'H',
                            'H', 'H', 'H', 'H', 'H', 'H', 'H','H', 'H', 'H', 'H', 'H'], True)

print(output)

## peak_merger(list_masses, list_abundances)
This function takes the two lists outputted by main_function, and merges the values in list_abundances if their values in list_masses are the same.

In [ ]:
import MASSiveChem.MASSiveChem as MC

output = MC.peak_merger([336.09, 334.094, 336.084, 334.088, 337.086, 335.087, 334.09, 335.095, 334.095, 334.097, 333.091, 334.09], 
                        [0.006155135615344779, 0.1389218494465703, 0.00025689925174544854, 0.005798234418254755, 0.00016431194999935186,
                        0.03458766547486355, 0.006161698124975695, 0.006260193058916763, 0.001252038611783352, 0.0019909461317594106, 
                        0.7806460744469201, 0.006245168595575361],
                        0.01)

print(output)

## Sulphur_nitrogen_adder(x_in, y_in, has_N, has_S, count_N, count_S)
This function takes the two outputted lists from peak_merger and adds the values for the sulphur and nitrogen peaks (if they need adding)

In [ ]:
import MASSiveChem.MASSiveChem as MC

x_in = [335.092, 335.086, 333.71807812500003, 334.09]
y_in = [0.14507698506191508, 0.006055133670000204, 0.8310629277992183, 0.006245168595575361]
has_N = False
has_S = True
count_N = 0
count_S = 1

output = MC.sulphur_nitrogen_adder(x_in, y_in, has_N, has_S, count_N, count_S)

print(output)

## peak_sorter(x_in, y_in)

This function takes the two lists outputted by the previous function, and sorts them in order for the values in x_in to be increasing, whilst keeping the elements in y_in associated to the correct x_iin values.

In [ ]:
import MASSiveChem.MASSiveChem as MC

x_in = [335.092, 335.086, 333.71807812500003, 334.09, 335.082]
y_in = [0.14507698506191508, 0.006055133670000204, 0.8310629277992183, 0.006245168595575361, 0.006245168595575361]

output = MC.peak_sorter(x_in, y_in)

print(output)

## delta_function_plotter(x_in, y_in)

This function takes the two lists from the previous function, and then adds values of 0 on the x axis around each point, in effect causing the points to form a Dirac delta function when plotted with a simple line connecting the points.

In [ ]:
import MASSiveChem.MASSiveChem as MC

x_in = [333.71807812500003, 334.09, 335.082, 335.086, 335.092]
y_in = [0.8310629277992183, 0.006245168595575361, 0.006245168595575361, 0.006055133670000204, 0.14507698506191508]

output = MC.delta_function_plotter(x_in, y_in)

print(output)

## double_plot(x_in,y_in)

This function takes as an input the list of the masses (x_in) and the list of their intensities (y_in) and plot them on a bokeh interface. The graph is interactive to be able to zoom on peaks.

In [ ]:
import MASSiveChem.MASSiveChem as MC
from bokeh.plotting import output_notebook, show

output_notebook() #not necessary if you want to plot the image outside the outbook

x_in = [333.21807812500003, 333.71807812500003, 333.71807812500003, 333.71807812500003, 334.09, 334.09, 334.09, 335.082, 335.082, 335.082, 335.086, 335.086, 335.086, 335.092, 335.092, 335.092, 336.092]
y_in = [0, 0, 0.8310629277992183, 0, 0, 0.006245168595575361, 0, 0, 0.006245168595575361, 0, 0, 0.006055133670000204, 0, 0, 0.14507698506191508, 0, 0]

output = MC.double_plot(x_in,y_in)

show(output)

## functional_group_finder(mol_smi)
This function takes in the smiles of a given molecule, and outputs a list of all of the contained functional groups (the list contains the names of the functional groups)

In [ ]:
import MASSiveChem.MASSiveChem as MC

mol_smi = 'CC1(C(N2C(S1)C(C2=O)NC(=O)CC3=CC=CC=C3)C(=O)O)C'

output = MC.functional_group_finder(mol_smi)

print(output)

## functional_group_display(contained_functional_groups)

This function takes as an input the list of the contained functional groups of the molecule and plots a table of the functional groups in a bokeh interface

In [ ]:
import MASSiveChem.MASSiveChem as MC
from bokeh.plotting import output_notebook, show

output_notebook() #not necessary if you want to plot the image outside the

functional_groups = ['Carboxylic Acid', 'Amide', 'Amide', 'Amine', 'Sulfide', 'Benzene']

output = MC.functional_group_display(functional_groups)

show(output)

## mol_web_show(mol_smi, show_Hs=False, show_3D = False)

This function takes as an input the SMILEs of the molecule and plots the image of the molecule in a bokeh interface

In [ ]:
import MASSiveChem.MASSiveChem as MC
from bokeh.plotting import output_notebook, show

output_notebook() #not necessary if you want to plot the image outside the notebook

mol_smi = 'CC1(C(N2C(S1)C(C2=O)NC(=O)CC3=CC=CC=C3)C(=O)O)C'

output = MC.mol_web_show(mol_smi)

show(output)

## all_in_one(p1, p2, p3)

This function takes as an input 3 bokeh plots and put them together in one bokeh interface NOT READY

In [ ]:
import MASSiveChem.MASSiveChem as MC
from bokeh.plotting import output_notebook, show

output_notebook() #not necessary if you want to plot the image outside the outbook

mol_smi = 'CC1(C(N2C(S1)C(C2=O)NC(=O)CC3=CC=CC=C3)C(=O)O)C'
functional_groups = ['Carboxylic Acid', 'Amide', 'Amide', 'Amine', 'Sulfide', 'Benzene']

x_in = [333.21807812500003, 333.71807812500003, 333.71807812500003, 333.71807812500003, 334.09, 334.09, 334.09, 335.082, 335.082, 335.082, 335.086, 335.086, 335.086, 335.092, 335.092, 335.092, 336.092]
y_in = [0, 0, 0.8310629277992183, 0, 0, 0.006245168595575361, 0, 0, 0.006245168595575361, 0, 0, 0.006055133670000204, 0, 0, 0.14507698506191508, 0, 0]

p1 = MC.double_plot(x_in,y_in)
p2 = MC.mol_web_show(mol_smi)
p3 = MC.functional_group_display(functional_groups)

output = MC.all_in_one(p1,p2,p3)

show(output)

## smiles_to_3D_plot(mol_smi)

This function takes as an input the SMILEs of a molecule and plots it in 3D in an interactive graph

In [ ]:
import MASSiveChem.MASSiveChem as MC

mol_smi = 'CC1(C(N2C(S1)C(C2=O)NC(=O)CC3=CC=CC=C3)C(=O)O)C'

output =MC.smiles_to_3D_plot(mol_smi)

output

## Full function

Now that these small constituant functions have been layed out, we can see how the main function, MC.spectrum, is formed. The functions above, when combined in the correct order, create the function called MC.spectrum. This looks something like this:

In [2]:
import MASSiveChem.MASSiveChem as MC
from bokeh.plotting import output_notebook, show

output_notebook() #not necessary if you want to plot the image outside the outbook

mol_smi = 'CC1(C(N2C(S1)C(C2=O)NC(=O)CC3=CC=CC=C3)C(=O)O)C'
imprecision = True
apparatus_resolution = 0.01

mol = MC.SMILEs_interpreter(mol_smi)

list_atoms = MC.molecule_list_generator(mol)

list_atoms_postionisation = MC.ionisation_method(list_atoms)

has_N, has_S, count_N, count_S = MC.sulphur_nitrogen_finder(list_atoms_postionisation)

list_masses, list_abundances = MC.main_function(list_atoms_postionisation, imprecision)

list_masses, list_abundances = MC.peak_merger(list_masses, list_abundances, apparatus_resolution)

list_masses, list_abundances = MC.sulphur_nitrogen_adder(list_masses, list_abundances, has_N, has_S, count_N, count_S)

list_masses_sorted, list_abundances_sorted = MC.peak_sorter(list_masses, list_abundances)

list_masses_plot, list_abundances_plot = MC.delta_function_plotter(list_masses_sorted, list_abundances_sorted)

spectrum = MC.double_plot(list_masses_plot,list_abundances_plot)

functional_groups_contained = MC.functional_group_finder(mol_smi)

functional_groups_plot = MC.functional_group_display(functional_groups_contained)

molecule_plot = MC.mol_web_show(mol_smi)

final_plot = MC.all_in_one(spectrum, molecule_plot, functional_groups_plot)

show(final_plot)

Loading BokehJS ...

here



## spectrum(mol_smi, imprecision_True_False, apparatus_resolution)
Running all these functions can be done using 1 single function called spectrum which can be run as the following:

In [ ]:
import MASSiveChem.MASSiveChem as MC
from bokeh.plotting import show

mol_smi = 'CC1(C(N2C(S1)C(C2=O)NC(=O)CC3=CC=CC=C3)C(=O)O)C'
imprecision = True
apparatus_resolution = 0.01

spectrum = MC.spectrum(mol_smi, imprecision, apparatus_resolution)

show(spectrum)

Finally, if you want to add the interactive 3D plot of the molecule you can use the following function

## spectrum_3D(mol_smi, imprecision_True_False, apparatus_resolution)

The function takes the same inputs as spectrum but adds the 3D interactive plot to the spectrum

In [ ]:
import MASSiveChem.MASSiveChem as MC

mol_smi = 'CC1(C(N2C(S1)C(C2=O)NC(=O)CC3=CC=CC=C3)C(=O)O)C'
imprecision = True
apparatus_resolution = 0.01

spectrum = MC.spectrum_3D(mol_smi, imprecision, apparatus_resolution)

spectrum.show()

Thanks for reading the notebook ! Hope you find it useful and interesting.